In [1]:
import rebound as rb
import reboundx as rbx
import astropy.units as u
import astropy.constants as constants
import numpy as np
import matplotlib.pyplot as plt

In [2]:
sim = rb.Simulation()
sim.units = ('yr', 'AU', 'Msun')
sim.add(m = 1)
sim.add(m = 1.e-4, a=0.5, e=0, inc = 0)
sim.move_to_com()

In [4]:
simx = rbx.Extras(sim)
mig = simx.load_force("type_I_migration")
simx.add_force(mig)

mig.params["inner_disk_edge_position"] = 0.1
mig.params["disk_edge_width"] = 0.03*(0.1**0.25)                                                    #hedge = 0.03 x (dedge^0.25)
mig.params["flaring_index"] = 0.25
mig.params["surface_density_exponent"] = 1
mig.params["initial_surface_density"] = ((1000* u.g /u.cm**2).to(u.Msun/u.AU**2)).value             #transformed from g/cm^2 to code units
mig.params["scale_height"] = 0.03
ps = sim.particles

In [ ]:
sim.integrator = 'whfast'
sim.dt = np.sqrt(0.1**3)/20

In [ ]:
times = np.linspace(0, 1e4, 1000)
a_integration = np.zeros((1000))
for i, t in enumerate(times):
    sim.integrate(t)
    a_integration[i] = ps[1].a

In [ ]:
G = (4*np.pi**2)                                                #In the code units used here
m = (1e-4)                                                      #Mass of the planets
h0 = (0.03)                                                     #Scale height (at 1AU)
sd0 = (((1000* u.g /u.cm**2).to(u.Msun/u.AU**2)).value)         #The surface density at 1 AU

tau_tilde = ((1*(h0**2)) / (3.8*m*sd0*(np.sqrt(G))))            #The terms of the equation for the semi-major axis independent on a

a_analytical = []
for i in (1 - (t/tau_tilde)):
    if i > 0.1:
        a_analytical.append(i)
    else:
        a_analytical.append(0.1)                                #Manually setting the analytically calculated a to stop at the inner disk edge as the integration does

In [ ]:
plt.plot(times*0.001, a_integration, label = 'Numerical evolution', c = 'green') 
plt.plot(times*0.001, a_analytical, label = 'Analytical evolution', c = 'brown')
plt.xlim(np.min(times)*0.001, np.max(times)*0.001) 
plt.xlabel('time [kyr]')
plt.ylabel('Semi-major axis [AU]')
plt.legend()